# LoveMemory AI - Анализ синтетических данных

Этот notebook анализирует сгенерированные синтетические данные для понимания распределений и паттернов.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path

# Настройка стиля графиков
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Загружаем синтетические данные
data_dir = Path('../data/synthetic_v1')

print("📊 Загружаем синтетические данные...")
users_df = pd.read_csv(data_dir / 'users.csv')
pairs_df = pd.read_csv(data_dir / 'pairs.csv')
interactions_df = pd.read_csv(data_dir / 'interactions.csv')
products_df = pd.read_csv(data_dir / 'product_catalog.csv')

print(f"✅ Загружено:")
print(f"  - Пользователей: {len(users_df)}")
print(f"  - Пар: {len(pairs_df)}")
print(f"  - Взаимодействий: {len(interactions_df)}")
print(f"  - Товаров: {len(products_df)}")


## 1. Анализ архетипов пользователей


In [ ]:
# Распределение архетипов
archetype_counts = users_df['archetype'].value_counts()
print("🎭 Распределение архетипов:")
print(archetype_counts)

# Визуализация
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
archetype_counts.plot(kind='bar', color='skyblue')
plt.title('Распределение архетипов пользователей')
plt.xlabel('Архетип')
plt.ylabel('Количество')
plt.xticks(rotation=45)

# Распределение возрастов по архетипам
plt.subplot(1, 2, 2)
users_df.boxplot(column='age', by='archetype', ax=plt.gca())
plt.title('Распределение возрастов по архетипам')
plt.xlabel('Архетип')
plt.ylabel('Возраст')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


## 2. Анализ языков любви
 

In [ ]:
# Анализ языков любви
love_languages = ['quality_time', 'physical_touch', 'words_of_affirmation', 'acts_of_service', 'receiving_gifts']

# Создаем DataFrame для языков любви
love_lang_data = []
for _, user in users_df.iterrows():
    for lang in love_languages:
        love_lang_data.append({
            'user_id': user['id'],
            'archetype': user['archetype'],
            'love_language': lang,
            'value': user['love_languages'][lang] if isinstance(user['love_languages'], dict) else 0.2
        })

love_lang_df = pd.DataFrame(love_lang_data)

# Визуализация языков любви по архетипам
plt.figure(figsize=(15, 10))

# Heatmap языков любви по архетипам
pivot_data = love_lang_df.pivot_table(
    values='value', 
    index='archetype', 
    columns='love_language', 
    aggfunc='mean'
)

plt.subplot(2, 2, 1)
sns.heatmap(pivot_data, annot=True, cmap='YlOrRd', fmt='.2f')
plt.title('Средние значения языков любви по архетипам')

# Распределение языков любви
plt.subplot(2, 2, 2)
love_lang_means = love_lang_df.groupby('love_language')['value'].mean()
love_lang_means.plot(kind='bar', color='lightcoral')
plt.title('Средние значения языков любви')
plt.xlabel('Язык любви')
plt.ylabel('Среднее значение')
plt.xticks(rotation=45)

# Boxplot языков любви
plt.subplot(2, 2, 3)
love_lang_df.boxplot(column='value', by='love_language', ax=plt.gca())
plt.title('Распределение языков любви')
plt.xlabel('Язык любви')
plt.ylabel('Значение')
plt.xticks(rotation=45)

# Топ языки любви по архетипам
plt.subplot(2, 2, 4)
for archetype in users_df['archetype'].unique():
    archetype_data = love_lang_df[love_lang_df['archetype'] == archetype]
    top_lang = archetype_data.groupby('love_language')['value'].mean().idxmax()
    print(f"{archetype}: {top_lang}")

plt.tight_layout()
plt.show()
